In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras

In [3]:
!pip install tensorflow

In [4]:
!pip install Pillow

In [5]:
from keras.preprocessing.image import ImageDataGenerator


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
test_datagen=ImageDataGenerator(rescale=1./255)

In [8]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QQr-GUQPXcQ-3ZnkE_sgPBAccTbKgwmJ2sxD1NQqU0gP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ibmfruitdisease-donotdelete-pr-vrmab2fmmvdelj'
object_key = 'Dataset Plant Disease.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [9]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
        unzip.extract(path)

In [10]:
pwd

'/home/wsuser/work'

In [11]:
import os 
filenames=os.listdir('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train')
filenames

['Apple___healthy',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Peach___healthy',
 'Peach___Bacterial_spot',
 'Apple___Black_rot']

In [12]:
x_train=train_datagen.flow_from_directory("/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train",target_size=(128,128),batch_size=24,class_mode='categorical')
x_test=test_datagen.flow_from_directory("/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test",target_size=(128,128),batch_size=24,class_mode='categorical')

Found 5384 images belonging to 6 classes.
Found 1686 images belonging to 6 classes.


In [13]:
model=Sequential()

In [14]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))


In [15]:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

In [17]:
model.add(Dense(6,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [18]:
len(x_train)

225

In [19]:
model.fit(x_train,steps_per_epoch=len(x_train),epochs=3,validation_data=x_test,validation_steps=len(x_train))

Epoch 1/3
225/225 [==============================] - 155s 687ms/step - loss: 1.0916 - accuracy: 0.7938 - val_loss: 0.2221 - val_accuracy: 0.9253
Epoch 2/3
225/225 [==============================] - 142s 629ms/step - loss: 0.2360 - accuracy: 0.9231
Epoch 3/3
225/225 [==============================] - 145s 643ms/step - loss: 0.1866 - accuracy: 0.9348


In [20]:
model.save('fruitdata.h5')

In [21]:
!tar -zcvf Fruit_Disease_Prediction_new.tgz fruitdata.h5

fruitdata.h5


In [22]:
ls 

'Dataset Plant Disease'/   fruitdata.h5   Fruit_Disease_Prediction_new.tgz


In [23]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 17.3 MB/s eta 0:00:01


In [24]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"4MLJtCFoXo24cER7G3Y-gc0-F3tRkvR-nqmOeNoOluOg"
}
client=APIClient(wml_credentials)

In [25]:
client=APIClient(wml_credentials)

In [26]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [27]:
space_uid = guid_from_space_name(client, 'FruitDisease')
print("Space UID = " + space_uid)

Space UID = f24981b2-cb82-49f1-8aa8-dc81f9ef3474


In [28]:
client.set.default_space(space_uid)

'SUCCESS'

In [29]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [35]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [36]:
 model_details = client.repository.store_model(model='Fruit_Disease_Prediction_new.tgz',meta_props={
 client.repository.ModelMetaNames.NAME:"CNN",
 client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                          )


In [37]:
model_id=client.repository.get_model_uid(model_details)
print(model_id)

This method is deprecated, please use get_model_id()
445f2540-582b-43d6-961c-004ed12e2714


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [38]:
client.repository.download(model_id, 'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [39]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [40]:
model=load_model('fruitdata.h5')


In [59]:
from io import BytesIO
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QQr-GUQPXcQ-3ZnkE_sgPBAccTbKgwmJ2sxD1NQqU0gP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ibmfruitdisease-donotdelete-pr-vrmab2fmmvdelj'
object_key = 'fruit_img.JPG'

streaming_body_7 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
img=image.load_img(streaming_body_7,target_size=(128,128))

TypeError: expected str, bytes or os.PathLike object, not StreamingBody